In [2]:
import SimulateData

In [1]:
config = {
    "signatures_file_path":     'cosmic_signatures/COSMIC_v3.4_SBS_GRCh37.txt',
    "signatures_to_extract":    ['SBS4', 'SBS6', 'SBS7a','SBS9','SBS18'],
    "n_samples":                10,
    "save_dir":                 'simulated_data',
    'sample_signature_distribution': {
        'distribution':             'uniform',
        'min':                      0.5,
        'max':                      2,
        'use_sign_active_prob':     True,
        'sign_active_prob':         0.4,    # Only used if use_sign_active_prob is True
        'n_sign_active':            2       # Only used if use_sign_active_prob is False
    },
    'noise_distribution': {
        'distribution':             'poisson',
        'avg_perc':                 0.05,
    },
    'counts_distribution': {
        'distribution':             'logscale',
        'min':                      1000,
        'max':                      50000,
    }
}

simulated_data, data_file, config_file = SimulateData.simulate_data(config)
print('')
print(simulated_data.head())

NameError: name 'SimulateData' is not defined

In [6]:
config = {
    "signatures_file_path":     'cosmic_signatures/COSMIC_v3.4_SBS_GRCh37.txt',
    "signatures_to_extract":    ['SBS4', 'SBS6', 'SBS7a','SBS9','SBS18'],
    "n_samples":                50,
    "save_dir":                 'simulated_data/experiment_noise_1',
    'sample_signature_distribution': {
        'distribution':             'uniform',
        'min':                      0.5,
        'max':                      2,
        'use_sign_active_prob':     True,
        'sign_active_prob':         0.4,    # Only used if use_sign_active_prob is True
        'n_sign_active':            2       # Only used if use_sign_active_prob is False
    },
    'noise_distribution': {
        'distribution':             'poisson',
        'avg_perc':                 0.05,
    },
    'counts_distribution': {
        'distribution':             'logscale',
        'min':                      1000,
        'max':                      50000,
    }
}

# loop over all the variables to change
noise_distribution_avg_perc_list = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4]
files = []
for avg_perc in noise_distribution_avg_perc_list:
    config['noise_distribution']['avg_perc'] = avg_perc
    files.append(SimulateData.simulate_data(config)[1])

print(files)

Sucessfully saved simulated data in simulated_data/experiment_noise_1/data_v1_GRCh37_4_6_7a_9_18.csv
Sucessfully saved meta-data in simulated_data/experiment_noise_1/config_v1.json
Sucessfully saved simulated data in simulated_data/experiment_noise_1/data_v2_GRCh37_4_6_7a_9_18.csv
Sucessfully saved meta-data in simulated_data/experiment_noise_1/config_v2.json
Sucessfully saved simulated data in simulated_data/experiment_noise_1/data_v3_GRCh37_4_6_7a_9_18.csv
Sucessfully saved meta-data in simulated_data/experiment_noise_1/config_v3.json
Sucessfully saved simulated data in simulated_data/experiment_noise_1/data_v4_GRCh37_4_6_7a_9_18.csv
Sucessfully saved meta-data in simulated_data/experiment_noise_1/config_v4.json
Sucessfully saved simulated data in simulated_data/experiment_noise_1/data_v5_GRCh37_4_6_7a_9_18.csv
Sucessfully saved meta-data in simulated_data/experiment_noise_1/config_v5.json
Sucessfully saved simulated data in simulated_data/experiment_noise_1/data_v6_GRCh37_4_6_7a_9_1